In [1]:
import pandas as pd
import psycopg2 as pg
from setup import dbname,user, host, password,url
import sqlalchemy as sa

In [2]:
df =pd.read_csv("titanic.csv")

In [3]:
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [4]:
df = df.rename(index =str, columns = {"Survived":"survived","Pclas":"pclass","Name":"name","Sex":"sex","Age":"age",\
                                      "Siblings/Spouses Aboard": "siblings_spouses_aboard",\
                                      "Parents/Children Aboard":"parents_children_aboard","Fare":"fare"})
df.index.names=["index"]
df.head()

,survived,Pclass,name,sex,age,siblings_spouses_aboard,parents_children_aboard,fare
index,,,,,,,,
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [5]:
df.describe()

,survived,Pclass,age,siblings_spouses_aboard,parents_children_aboard,fare
count,887.000000,887.000000,887.000000,887.000000,887.000000,887.00000
mean,0.385569,2.305524,29.471443,0.525366,0.383315,32.30542
std,0.487004,0.836662,14.121908,1.104669,0.807466,49.78204
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.00000
25%,0.000000,2.000000,20.250000,0.000000,0.000000,7.92500
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.45420
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.13750
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.32920


In [47]:
pg_conn = pg.connect(dbname=dbname,user=user,host= host,password = password)

In [48]:
pg_conn

<connection object at 0x7fe8e96f7048; dsn: 'user=uriewcad password=xxx dbname=uriewcad host=stampy.db.elephantsql.com', closed: 0>

In [49]:
create_titanic_table = """
CREATE TYPE pclass AS ENUM ('1','2','3');
CREATE TYPE gender AS ENUM ('male','female');
CREATE TABLE titanic (
index SERIAL PRIMARY KEY,
survived boolean,
pclass pclass,
name varchar(75),
sex gender,
sge int,
siblings_spouse_aboard int,
parents_children_aboard int,
fare float
);
"""

In [56]:
create_titanic_table = """
CREATE TABLE titanic (
index SERIAL PRIMARY KEY,
survived boolean,
pclass pclass,
name varchar(75),
sex gender,
sge int,
siblings_spouse_aboard int,
parents_children_aboard int,
fare float
);
"""

In [18]:
#schema = {"index":}

In [57]:
pg_curs = pg_conn.cursor()

In [60]:
pg_curs.execute(create_titanic_table)

In [61]:
pg_conn.commit()

In [14]:
con = sa.create_engine(url)


In [15]:
df.to_csv("titanic1.csv")

In [69]:
df.to_sql(name="titanic",con=con, if_exists="append")
#dtype={"index":Integer, "survived":Boolean(), }

In [70]:
pg_conn.commit()

InterfaceError: connection already closed

In [72]:
pg_conn = pg.connect(dbname=dbname,user=user,host= host,password = password)
pg_curs = pg_conn.cursor()

In [73]:
pg_curs,pg_conn

(<cursor object at 0x7fe8e9783618; closed: 0>,
 <connection object at 0x7fe8e9ae9d58; dsn: 'user=uriewcad password=xxx dbname=uriewcad host=stampy.db.elephantsql.com', closed: 0>)

In [74]:
#results = pg_curs.execute('SELECT index FROM titanic1 LIMIT 20;')
results = pg_curs.execute('SELECT * FROM titanic LIMIT 20;')

In [75]:
pg_curs.fetchall()

[('0', 0, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 1, 0, 7.25),
 ('1',
  1,
  1,
  'Mrs. John Bradley (Florence Briggs Thayer) Cumings',
  'female',
  38.0,
  1,
  0,
  71.2833),
 ('2', 1, 3, 'Miss. Laina Heikkinen', 'female', 26.0, 0, 0, 7.925),
 ('3',
  1,
  1,
  'Mrs. Jacques Heath (Lily May Peel) Futrelle',
  'female',
  35.0,
  1,
  0,
  53.1),
 ('4', 0, 3, 'Mr. William Henry Allen', 'male', 35.0, 0, 0, 8.05),
 ('5', 0, 3, 'Mr. James Moran', 'male', 27.0, 0, 0, 8.4583),
 ('6', 0, 1, 'Mr. Timothy J McCarthy', 'male', 54.0, 0, 0, 51.8625),
 ('7', 0, 3, 'Master. Gosta Leonard Palsson', 'male', 2.0, 3, 1, 21.075),
 ('8',
  1,
  3,
  'Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson',
  'female',
  27.0,
  0,
  2,
  11.1333),
 ('9',
  1,
  2,
  'Mrs. Nicholas (Adele Achem) Nasser',
  'female',
  14.0,
  1,
  0,
  30.0708),
 ('10', 1, 3, 'Miss. Marguerite Rut Sandstrom', 'female', 4.0, 1, 1, 16.7),
 ('11', 1, 1, 'Miss. Elizabeth Bonnell', 'female', 58.0, 0, 0, 26.55),
 ('12', 0, 3, 'M

In [68]:
pg_conn.close()

In [45]:
pg_curs.close()

In [39]:
pg_curs.execute("DROP TABLE titanic;")

ProgrammingError: table "titanic" does not exist
